# Limpieza de datos de Macrofauna 2022

In [88]:
import numpy as np 
import pandas as pd
import re 
import datetime
import traceback as tb
import sys

df_macrofauna = pd.read_excel(r'C:\Users\felipe.abarzua\Desktop\DATA_AMBIENTALES\Proyecto Seguimiento Ambiental\2022_P656150\2_BD_MACROFAUNA_2022.xlsx' , sheet_name='BD_MACROF_2022_656-150')


In [89]:
df_macrofauna

,Nombre del Proyecto:,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICULTURA EN CHILE Y SU EFECTO EN LOS ECOSISTEMAS DE EMPLAZAMIENTO 2022 -2023,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,Codigo del Proyecto:,656-150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Etapa Proyecto:,Objetivo 1. Actividad 2. Muestreos y Analisis ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jefe Proyecto,Gastón Vidal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Institucion:,INSTITUTO DE FOMENTO PESQUERO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nombre Sector:,"Seno Reloncaví, Estuario de Reloncavi,Calbuco,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2700,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,Gastón Vidal,INSTITUTO DE FOMENTO PESQUERO,Natales,XII,Macrofauna,2023-01-10 00:00:00,2023-01-10 00:00:00,...,230,Van Veen,1,0.0041,10,0.041,Priapula,Priapula,Priapula,Priapula
2701,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,Gastón Vidal,INSTITUTO DE FOMENTO PESQUERO,Natales,XII,Macrofauna,2023-01-10 00:00:00,2023-01-10 00:00:00,...,230,Van Veen,2,0.0116,20,0.116,Annelida,Polychaeta,Ampharetidae,Melinna cf. cristata
2702,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,Gastón Vidal,INSTITUTO DE FOMENTO PESQUERO,Natales,XII,Macrofauna,2023-01-10 00:00:00,2023-01-10 00:00:00,...,230,Van Veen,3,0.0016,30,0.016,Annelida,Polychaeta,Cirratulidae,Apelochaeta sp.
2703,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,Gastón Vidal,INSTITUTO DE FOMENTO PESQUERO,Natales,XII,Macrofauna,2023-01-10 00:00:00,2023-01-10 00:00:00,...,230,Van Veen,1,0.0036,10,0.036,Annelida,Polychaeta,Nephtyidae,Aglaophamus sp.


In [142]:
def limpieza_macrofauna(df):

    try:

        #Eliminamos las filas innecesarias
        df = df.iloc[13: , :]

        #Reiniciamos los indices
        df = df.reset_index(drop = True)

        #Creamos una copia del dataset
        df_macrofauna_copy = df.copy()

        #Dejamos la primera fila como columnas 
        df_macrofauna_copy.columns = df_macrofauna_copy.iloc[0]

        #Eliminamos la primera fila 
        df_macrofauna_copy = df_macrofauna_copy.iloc[1: , :]

        #Reiniciamos los indices 
        df_macrofauna_copy = df_macrofauna_copy.reset_index(drop = True)

        #Creamos una columna ID 
        df_macrofauna_copy.insert(0 , 'ID' , df_macrofauna_copy.index + 1 )

        # Creamos la lista de columnas 
        columnas = df_macrofauna_copy.columns.to_list() 
        
        #Creamos las columnas fijas y las columnas variables
        
        columnas_fijas = ['ID', 'NOMB_PROY', 'COD_PROY', 'ETAP_PROY', 'JEFE_PROY', 'INST_PROY',
       'NOM_SECTOR', 'COD_REG', 'VARIABLE', 'FECHA_INI', 'FECHA_TER',
       'N_ESTACION', 'LATITUD', 'LONGUITUD', 'PROF_SECT',  'PHYLUM', 'CLASE', 'FAMILIA', 'GEN/ESPECIE']
        
        columnas_variables = ['ABUNDANCIA', 'BIOMASA', 'IND_M2', 'GRS_M2' , 'N°_TAXA', 'NUM_EJEMPL', 'TIPO_DRAGA']

        #Realizamos un melt para convertir las columnas variables en filas 

        df_macrofauna_copy = pd.melt(df_macrofauna_copy , id_vars= columnas_fijas , value_vars= columnas_variables , var_name= 'VARIABLE_MUESTREO' , value_name='VALOR' )

        #Cambiamos los nombres de las columna de VARIABLE_MUESTREO

        df_macrofauna_copy['VARIABLE_MUESTREO'] = np.where(df_macrofauna_copy['VARIABLE_MUESTREO'] == 'ABUNDANCIA' , 'ABUNDANCIA TOTAL' , df_macrofauna_copy['VARIABLE_MUESTREO'])
        df_macrofauna_copy['VARIABLE_MUESTREO'] = np.where(df_macrofauna_copy['VARIABLE_MUESTREO'] == 'BIOMASA' , 'BIOMASA TOTAL' , df_macrofauna_copy['VARIABLE_MUESTREO'])
        df_macrofauna_copy['VARIABLE_MUESTREO'] = np.where(df_macrofauna_copy['VARIABLE_MUESTREO'] == 'NUM_EJEMPL' , 'NUMERO EJEMPLARES' , df_macrofauna_copy['VARIABLE_MUESTREO'])
        df_macrofauna_copy['VARIABLE_MUESTREO'] = np.where(df_macrofauna_copy['VARIABLE_MUESTREO'] == 'TIPO_DRAGA' , 'TIPO DRAGA' , df_macrofauna_copy['VARIABLE_MUESTREO'])
        df_macrofauna_copy['VARIABLE_MUESTREO'] = np.where(df_macrofauna_copy['VARIABLE_MUESTREO'] == 'N°_TAXA' , 'NUMERO DE TAXA' , df_macrofauna_copy['VARIABLE_MUESTREO'])
        

        #Modificamos el NOMB_PROY
        df_macrofauna_copy['NOMB_PROY'] = 'ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICULTURA EN CHILE Y SU EFECTO EN LOS ECOSISTEMAS DE EMPLAZAMIENTO 2022-2023'

        #Renombramos las columnas 
        df_macrofauna_copy = df_macrofauna_copy.rename(columns={'GEN/ESPECIE' : 'TAXA'})

        #Modificamos el formato de FECHA_INICIO y FECHA_TERMINO

        df_macrofauna_copy['FECHA_INI'] = df_macrofauna_copy['FECHA_INI'].dt.strftime('%d/%m/%Y %H:%M:%S')
        df_macrofauna_copy['FECHA_TER'] = df_macrofauna_copy['FECHA_TER'].dt.strftime('%d/%m/%Y %H:%M:%S')
    

        return df_macrofauna_copy , columnas , columnas_fijas , columnas_variables
    

    except Exception as e:
        print(f"Ocurrió un error en la función limpieza_macrofauna: {e}")
        ex = sys.exc_info()[2]
        print("Error en el archivo : " + str(ex.tb_frame) + " En la linea : " + str(ex.tb_lineno))
        
    return None #O devuelve un valor que indique el fallo

In [143]:
df_macrofauna_copy , columnas , columnas_fijas , columnas_variables = limpieza_macrofauna(df_macrofauna)

In [145]:
df_macrofauna_copy['FECHA_TER'].unique()

array(['27/10/2022 00:00:00', '24/10/2022 00:00:00',
       '09/08/2022 00:00:00', '26/10/2022 00:00:00',
       '25/10/2022 00:00:00', '10/08/2022 00:00:00',
       '11/08/2022 00:00:00', '12/08/2022 00:00:00',
       '15/08/2022 00:00:00', '14/08/2022 00:00:00',
       '16/08/2022 00:00:00', '21/10/2022 00:00:00',
       '20/08/2022 00:00:00', '18/08/2022 00:00:00',
       '23/10/2022 00:00:00', '22/10/2022 00:00:00',
       '20/10/2022 00:00:00', '21/08/2022 00:00:00',
       '28/09/2022 00:00:00', '27/09/2022 00:00:00',
       '26/09/2022 00:00:00', '29/09/2022 00:00:00',
       '19/10/2022 00:00:00', '10/10/2022 00:00:00',
       '08/10/2022 00:00:00', '09/10/2022 00:00:00',
       '04/10/2022 00:00:00', '06/10/2022 00:00:00',
       '02/10/2022 00:00:00', '30/09/2022 00:00:00',
       '11/01/2023 00:00:00', '10/01/2023 00:00:00',
       '19/01/2023 00:00:00', '18/01/2023 00:00:00'], dtype=object)

In [131]:
df_macrofauna_copy['VARIABLE_MUESTREO'].unique()

array(['ABUNDANCIA TOTAL', 'BIOMASA TOTAL', 'IND_M2', 'GRS_M2',
       'NUMERO DE TAXA', 'NUMERO EJEMPLARES', 'TIPO DRAGA'], dtype=object)

In [137]:
df_macrofauna_copy.query("ID == 2")

,ID,NOMB_PROY,COD_PROY,ETAP_PROY,JEFE_PROY,INST_PROY,NOM_SECTOR,COD_REG,VARIABLE,FECHA_INI,...,N_ESTACION,LATITUD,LONGUITUD,PROF_SECT,PHYLUM,CLASE,FAMILIA,TAXA,VARIABLE_MUESTREO,VALOR
1,2,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,Gastón Vidal,INSTITUTO DE FOMENTO PESQUERO,Estuario,X,Macrofauna,2022-10-27,...,1,-41.437472,-72.296639,28,Mollusca,Bivalvia,Tellinidae,Macoploma inornata,ABUNDANCIA TOTAL,27
2692,2,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,Gastón Vidal,INSTITUTO DE FOMENTO PESQUERO,Estuario,X,Macrofauna,2022-10-27,...,1,-41.437472,-72.296639,28,Mollusca,Bivalvia,Tellinidae,Macoploma inornata,BIOMASA TOTAL,0.2032
5383,2,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,Gastón Vidal,INSTITUTO DE FOMENTO PESQUERO,Estuario,X,Macrofauna,2022-10-27,...,1,-41.437472,-72.296639,28,Mollusca,Bivalvia,Tellinidae,Macoploma inornata,IND_M2,270
8074,2,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,Gastón Vidal,INSTITUTO DE FOMENTO PESQUERO,Estuario,X,Macrofauna,2022-10-27,...,1,-41.437472,-72.296639,28,Mollusca,Bivalvia,Tellinidae,Macoploma inornata,GRS_M2,2.032
10765,2,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,Gastón Vidal,INSTITUTO DE FOMENTO PESQUERO,Estuario,X,Macrofauna,2022-10-27,...,1,-41.437472,-72.296639,28,Mollusca,Bivalvia,Tellinidae,Macoploma inornata,NUMERO DE TAXA,31
13456,2,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,Gastón Vidal,INSTITUTO DE FOMENTO PESQUERO,Estuario,X,Macrofauna,2022-10-27,...,1,-41.437472,-72.296639,28,Mollusca,Bivalvia,Tellinidae,Macoploma inornata,NUMERO EJEMPLARES,4910
16147,2,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,Gastón Vidal,INSTITUTO DE FOMENTO PESQUERO,Estuario,X,Macrofauna,2022-10-27,...,1,-41.437472,-72.296639,28,Mollusca,Bivalvia,Tellinidae,Macoploma inornata,TIPO DRAGA,Van Veen


In [146]:
df_macrofauna_copy.to_excel('output/2022/AMBIENTALES_MACROFAUNA_2022.xlsx' , index= False)

# Limpieza Muestreo Regular

In [5]:
df_muestreo_regular = pd.read_excel(r'C:\Users\felipe.abarzua\Desktop\DATA_AMBIENTALES\Proyecto Seguimiento Ambiental\2022_P656150\1_BD_MuestreoRegular_2022.xlsx' , sheet_name = 'BD_Muestreo_OBJ_IFOP2022')

In [6]:
df_muestreo_regular

,Nombre del Proyecto:,"ESTUDIO DEL DESEMPEÑO AMBIENTAL DE LA ACUICULTURA EN CHILE Y SU EFECTO EN LOS ECOSISTEMAS DE EMPLAZAMIENTO, 2022-2023.",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39
0,Codigo del Proyecto:,656-150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Etapa Proyecto:,Objetivo 1. Actividad 2. Muestreos y Analisis ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jefe Proyecto,Gastón Vidal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Institución:,INSTITUTO DE FOMENTO PESQUERO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nombre Sector:,"Seno Reloncaví, Estuario de Reloncavi, Calbuco...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,ESTUDIO DEL DESEMPEÑO AMBIENTAL DE LA ACUICULT...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,GASTON VIDAL,INSTITUTO DE FOMENTO PESQUERO,Natales,X,"Ph, Temperatura (°C), Redox EHNHE (mV), Sulfur...",2023-01-19 00:00:00,2023-01-19 00:00:00,...,2.452311,18920,18,10.91,0.1101,0.08774,1.5,Ligeramente perturbado,"Conchilla, restos organicos, arena fina",45
156,ESTUDIO DEL DESEMPEÑO AMBIENTAL DE LA ACUICULT...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,GASTON VIDAL,INSTITUTO DE FOMENTO PESQUERO,Natales,X,"Ph, Temperatura (°C), Redox EHNHE (mV), Sulfur...",2023-01-18 00:00:00,2023-01-18 00:00:00,...,3.402234,300,12,1.084,0.858,0.795,2,Ligeramente perturbado,Arena fina (café oscuro),48A
157,ESTUDIO DEL DESEMPEÑO AMBIENTAL DE LA ACUICULT...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,GASTON VIDAL,INSTITUTO DE FOMENTO PESQUERO,Natales,X,"Ph, Temperatura (°C), Redox EHNHE (mV), Sulfur...",2023-01-18 00:00:00,2023-01-18 00:00:00,...,5.839302,sin_macrof,sin_macrof,sin_macrof,sin_macrof,sin_macrof,sin_macrof,sin_macrof,"Arena fina fango negro, conchilla c/olor",48B
158,ESTUDIO DEL DESEMPEÑO AMBIENTAL DE LA ACUICULT...,656-150,Objetivo 1. Actividad 2. Muestreos y Analisis ...,GASTON VIDAL,INSTITUTO DE FOMENTO PESQUERO,Natales,X,"Ph, Temperatura (°C), Redox EHNHE (mV), Sulfur...",2023-01-18 00:00:00,2023-01-18 00:00:00,...,3.920293,130,9,0.817,0.9112,0.9549,2.125,Ligeramente perturbado,Arena fina café,47A


In [69]:
def limpieza_muestreo_regular(df):

    try:

        # Eliminamos las filas innecesarias
        df = df.iloc[18: , :]

        #Reiniciamos los indices
        df = df.reset_index(drop = True)

        #Creamos una copia del dataset
        df_muestreo_regular_copy = df.copy()

        #Dejamos la primera fila como columnas 
        df_muestreo_regular_copy.columns = df_muestreo_regular_copy.iloc[0]

        #Eliminamos la primera fila 
        df_muestreo_regular_copy = df_muestreo_regular_copy.iloc[1: , :]

        #Reiniciamos los indices
        df_muestreo_regular_copy = df_muestreo_regular_copy.reset_index(drop=True)

        #Creamos una columna llamada ID 
        df_muestreo_regular_copy.insert(0 , 'ID' , df_muestreo_regular_copy.index + 1)

        
        #Renombramos las columnas de NOM_SECTOR 
        df_muestreo_regular_copy.columns.values[6] = 'NOM_SECTOR_1'
        df_muestreo_regular_copy.columns.values[18] = 'NOM_SECTOR_2'

        #Eliminamos la columna de NOM_SECTOR_2 que está de más
        df_muestreo_regular_copy.drop(columns=['NOM_SECTOR_2'], inplace=True)

       #Renombramos la columna NOM_SECTOR_1 a NOM_SECTOR

        df_muestreo_regular_copy.columns.values[6] = 'NOM_SECTOR'


        #Creamos la lista de columnas 
        columnas = df_muestreo_regular_copy.columns.to_list()

        
        #Creamos las columnas fijas y Variables
        columnas_fijas = ['ID', 'NOMB_PROY', 'COD_PROY', 'ETAP_PROY', 'JEFE_PROY', 'INST_PROY',
       'NOM_SECTOR', 'COD_REG', 'VARIABLE', 'FECHA_INI', 'FECHA_TER',
       'HORA_INI', 'HORA_FIN', 'N_ESTACION', 'LATITUD', 'LONGUITUD',
       'PROF_SECT' ,'OBSERV']

        columnas_variables = [ 'PROF_VAR', 'Ph_REPL_1', 'Ph_REPL_2', 'Ph_REPL_3',
       'TEMP_REPL_1', 'TEMP_REPL_2', 'TEMP_REPL_3', 'REDOX_REPL_1',
       'REDOX_REPL_2', 'REDOX_REPL_3', 'SULF_REPL_1', 'SULF_REPL_2',
       'SULF_REPL_3', 'MAT_ORG', 'ABUNDANCIA', 'N°_TAXA', 'BIOM_TOTAL',
       'DIV_SHANNON', 'UNIF_PIELOU', 'AMBI', 'CLASIF_PERTURB']


        #Realizamos un melt para convertir las columnas de variables en filas

        df_muestreo_regular_copy = pd.melt(df_muestreo_regular_copy , id_vars= columnas_fijas , value_vars= columnas_variables , var_name= 'VARIABLE_MUESTREO' , value_name='VALOR' )

        #Se van a eliminar todas las filas que la FECHA_INI es "sd" ya que no sirven para el proposito de la BD. Se eliminaron 20 registros
        df_muestreo_regular_copy = df_muestreo_regular_copy[df_muestreo_regular_copy['FECHA_INI'] != 'sd']

        #Cambiamos los tipos de FECHA_INI y FECHA_FIN a datetime

        df_muestreo_regular_copy['FECHA_INI'] = pd.to_datetime(df_muestreo_regular_copy['FECHA_INI'])
        df_muestreo_regular_copy['FECHA_TER'] = pd.to_datetime(df_muestreo_regular_copy['FECHA_TER'])

         #FECHA INI CONVERTIDA A STRING 
        df_muestreo_regular_copy['FECHA_INI_STR'] =  df_muestreo_regular_copy['FECHA_INI'].dt.strftime("%d/%m/%Y")
        df_muestreo_regular_copy['FECHA_FIN_STR'] = df_muestreo_regular_copy['FECHA_TER'].dt.strftime("%d/%m/%Y")

        #Reemplazamos los valores de FECHA_FIN de 'sd' POR "00:00"

        df_muestreo_regular_copy['HORA_FIN'] = np.where(df_muestreo_regular_copy['HORA_FIN'] == 'sd' , datetime.time(0,0) , df_muestreo_regular_copy['HORA_FIN'])
        
         # Realizamos una funcion lambda para poder combinar la FECHA_INI_STR con la hora y convertirlo a datetime

        df_muestreo_regular_copy['FECHA_INICIO'] = df_muestreo_regular_copy.apply(
        lambda row: datetime.datetime.strptime(row['FECHA_INI_STR'], "%d/%m/%Y").replace(
            hour=row['HORA_INI'].hour,
            minute=row['HORA_INI'].minute,
            second=row['HORA_INI'].second
            ),
            axis=1
        )

        

        #Realizamos una funcion lambda para conbinar la FECHA_FIN_STR con la HORA_FIN

        df_muestreo_regular_copy['FECHA_TERMINO'] = df_muestreo_regular_copy.apply(
        lambda row: datetime.datetime.strptime(row['FECHA_FIN_STR'], "%d/%m/%Y").replace(
            hour=row['HORA_FIN'].hour,
            minute=row['HORA_FIN'].minute,
            second=row['HORA_FIN'].second
            ),
            axis=1
        )

        
        #Aplicamos un UPPER a la columna de VARIABLE_MUESTREO

        df_muestreo_regular_copy['VARIABLE_MUESTREO'] = df_muestreo_regular_copy['VARIABLE_MUESTREO'].str.upper()

        #Cambiamos los nombres de las VARIABLE_MUESTREO

        df_muestreo_regular_copy['VARIABLE_MUESTREO'] = np.where(df_muestreo_regular_copy['VARIABLE_MUESTREO'] == 'MAT_ORG' , 'MATERIA ORGANICA' , df_muestreo_regular_copy['VARIABLE_MUESTREO'])
        df_muestreo_regular_copy['VARIABLE_MUESTREO'] = np.where(df_muestreo_regular_copy['VARIABLE_MUESTREO'] == 'ABUNDANCIA' , 'ABUNDANCIA TOTAL' , df_muestreo_regular_copy['VARIABLE_MUESTREO'])
        df_muestreo_regular_copy['VARIABLE_MUESTREO'] = np.where(df_muestreo_regular_copy['VARIABLE_MUESTREO'] == 'N°_TAXA' , 'NUMERO DE TAXA' , df_muestreo_regular_copy['VARIABLE_MUESTREO'])
        df_muestreo_regular_copy['VARIABLE_MUESTREO'] = np.where(df_muestreo_regular_copy['VARIABLE_MUESTREO'] == 'BIOM_TOTAL' , 'BIOMASA TOTAL' , df_muestreo_regular_copy['VARIABLE_MUESTREO'])
        df_muestreo_regular_copy['VARIABLE_MUESTREO'] = np.where(df_muestreo_regular_copy['VARIABLE_MUESTREO'] == 'DIV_SHANNON' , 'DISIDAD DE SHANNON' , df_muestreo_regular_copy['VARIABLE_MUESTREO'])
        df_muestreo_regular_copy['VARIABLE_MUESTREO'] = np.where(df_muestreo_regular_copy['VARIABLE_MUESTREO'] == 'UNIF_PIELOU' , 'UNIFORMIDAD DE PIELOU' , df_muestreo_regular_copy['VARIABLE_MUESTREO'])
        df_muestreo_regular_copy['VARIABLE_MUESTREO'] = np.where(df_muestreo_regular_copy['VARIABLE_MUESTREO'] == 'CLASIF_PERTURB' , 'CLASIFICACIÓN DE PERTURBACIÓN' , df_muestreo_regular_copy['VARIABLE_MUESTREO'])


        #Modificamos el formato de FECHA_INICIO y FECHA_TERMINO

        df_muestreo_regular_copy['FECHA_INICIO'] = df_muestreo_regular_copy['FECHA_INICIO'].dt.strftime('%d/%m/%Y %H:%M:%S')
        df_muestreo_regular_copy['FECHA_TERMINO'] = df_muestreo_regular_copy['FECHA_TERMINO'].dt.strftime('%d/%m/%Y %H:%M:%S')

        #Modificamos el NOMB_PROY

        df_muestreo_regular_copy['NOMB_PROY'] = 'ESTUDIO DEL DESEMPEÑO AMBIENTAL DE LA ACUICULTURA EN CHILE Y SU EFECTO EN LOS ECOSISTEMAS DE EMPLAZAMIENTO 2022-2023'

        #Modificamos el Valor cambiando los sd , sin_macrof por nan

        df_muestreo_regular_copy['VALOR'] = np.where(df_muestreo_regular_copy['VALOR'] == 'sd' , np.nan , df_muestreo_regular_copy['VALOR'])
        df_muestreo_regular_copy['VALOR'] = np.where(df_muestreo_regular_copy['VALOR'] == 'sin_macrof' , np.nan , df_muestreo_regular_copy['VALOR'])

        

        


        return df_muestreo_regular_copy , columnas , columnas_fijas , columnas_variables

    except Exception as e:
        print(f"Ocurrió un error en la función limpieza_macrofauna: {e}")
        ex = sys.exc_info()[2]
        print("Error en el archivo : " + str(ex.tb_frame) + " En la linea : " + str(ex.tb_lineno))
        
    return None #O devuelve un valor que indique el fallo



In [70]:
df_muestreo_regular_copy , columnas , columnas_fijas , columnas_variables  = limpieza_muestreo_regular(df_muestreo_regular)

In [71]:
df_muestreo_regular_copy.columns

Index(['ID', 'NOMB_PROY', 'COD_PROY', 'ETAP_PROY', 'JEFE_PROY', 'INST_PROY',
       'NOM_SECTOR', 'COD_REG', 'VARIABLE', 'FECHA_INI', 'FECHA_TER',
       'HORA_INI', 'HORA_FIN', 'N_ESTACION', 'LATITUD', 'LONGUITUD',
       'PROF_SECT', 'OBSERV', 'VARIABLE_MUESTREO', 'VALOR', 'FECHA_INI_STR',
       'FECHA_FIN_STR', 'FECHA_INICIO', 'FECHA_TERMINO'],
      dtype='object')

In [72]:
df_muestreo_regular_copy['VARIABLE_MUESTREO'].unique()

array(['PROF_VAR', 'PH_REPL_1', 'PH_REPL_2', 'PH_REPL_3', 'TEMP_REPL_1',
       'TEMP_REPL_2', 'TEMP_REPL_3', 'REDOX_REPL_1', 'REDOX_REPL_2',
       'REDOX_REPL_3', 'SULF_REPL_1', 'SULF_REPL_2', 'SULF_REPL_3',
       'MATERIA ORGANICA', 'ABUNDANCIA TOTAL', 'NUMERO DE TAXA',
       'BIOMASA TOTAL', 'DISIDAD DE SHANNON', 'UNIFORMIDAD DE PIELOU',
       'AMBI', 'CLASIFICACIÓN DE PERTURBACIÓN'], dtype=object)

In [81]:
df_muestreo_regular_copy.query("VARIABLE_MUESTREO == 'UNIFORMIDAD DE PIELOU' ")['VALOR'].unique()

array([0.6993, 0.6863, 0.8898, 0.8236, 0.6955, 0.7422, 0.9461, 0.7773,
       0.7506, 0.5093, 0.8524, 0.8478, 0.7915, 0.6155, 0.6615, 0.7665,
       0.6521, 0.84, 0.6228, 0.5688, 0.7722, 0.4353, 0.7321, 0.7839,
       0.7724, 0.8457, 0.392, 0.4711, 0.7286, 0.7627, 0.8768, 0.6699,
       0.5058, 0.7335, 0.7688, nan, 0.7394, 0.8275, 0.4796, 0.8081,
       0.7837, 0.6894, 0.7086, 0.6369, 0.622, 0.6117, 0.83, 0.6724,
       0.5496, 0.6587, 0.7587, 0.692, 0.7734, 0.6915, 0.6613, 0.8282,
       0.9518, 0.6244, 0.5638, 0.8032, 0.9013, 0.2554, 0.8512, 0.719,
       0.8189, 0.5898, 0.8025, 0.8925, 0.8132, 0.5937, 0.6426, 0.5737,
       0.9107, 0.3581, 0.7543, 0.7614, 0.4476, 0.5959, 0.7809, 0.6574,
       0.6684, 0.4898, 0.8561, 0.7109, 0.7494, 0.3987, 0.8552, 0.8301,
       0.7706, 0.8592, 0.8578, 0.8784, 0.7454, 0.4126, 0.8619, 0.6996,
       0.6629, 0.7984, 0.8862, 0.6419, 0.9427, 0.8462, 0.9425, 0.8068,
       0.7759, 0.8443, 0.8764, 0.8369, 0.8544, 0.8351, 0.9577, 0.8283,
       0.8657, 0.

In [85]:
df_muestreo_regular_copy['FECHA_TERMINO'].unique()

array(['27/10/2022 14:48:00', '27/10/2022 14:27:00',
       '27/10/2022 13:56:00', '27/10/2022 13:36:00',
       '27/10/2022 12:09:00', '27/10/2022 11:10:00',
       '27/10/2022 10:15:00', '27/10/2022 09:23:00',
       '27/10/2022 08:22:00', '24/10/2022 13:37:00',
       '24/10/2022 11:51:00', '09/08/2022 12:20:00',
       '09/08/2022 09:15:00', '26/10/2022 11:32:00',
       '26/10/2022 12:09:00', '26/10/2022 18:07:00',
       '26/10/2022 18:23:00', '25/10/2022 10:51:00',
       '10/08/2022 08:40:00', '10/08/2022 10:30:00',
       '10/08/2022 09:45:00', '09/08/2022 00:00:00',
       '10/08/2022 11:00:00', '10/08/2022 12:38:00',
       '10/08/2022 13:34:00', '10/08/2022 14:45:00',
       '11/08/2022 10:30:00', '11/08/2022 09:50:00',
       '10/08/2022 17:26:00', '10/08/2022 16:55:00',
       '12/08/2022 08:45:00', '12/08/2022 10:05:00',
       '12/08/2022 12:25:00', '12/08/2022 11:50:00',
       '11/08/2022 15:50:00', '11/08/2022 13:35:00',
       '11/08/2022 13:10:00', '15/08/2022 09:0

In [87]:
df_muestreo_regular_copy.to_excel('output/2022/AMBIENTALES_MUESTREO_REGULAR_2022.xlsx' , index=False)

# Limpieza de datos de CTD

In [10]:
df_ctd = pd.read_excel(r'C:\Users\felipe.abarzua\Desktop\DATA_AMBIENTALES\Proyecto Seguimiento Ambiental\2022_P656150\3_BD_CTD_2022.xlsx' , sheet_name='BD_CTD2022_656-150')

In [11]:
df_ctd

,Nombre del Proyecto:,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICULTURA EN CHILE Y SU EFECTO EN LOS ECOSISTEMAS DE EMPLAZAMIENTO 2022 -2023,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,Codigo del Proyecto:,656-150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Etapa Proyecto:,Objetivo 1. Actividad 2. Muestreos y Analisis ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jefe Proyecto,Gastón Vidal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Institución:,Instituto de Fomento Pesquero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sectores:,"Estuario de Reloncavi, Seno de Reloncavi, Golf...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6458,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,2-10-2022,2-10-2022,CTD SeaBird SBE 19plus V 2.5.2 SERIAL NO. 6900,-45.668,-73.299,32,224,228,...,79.163,231.355,30.0689,23.1376,NaN,NaN,X,32,20:23,12
6459,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,2-10-2022,2-10-2022,CTD SeaBird SBE 19plus V 2.5.2 SERIAL NO. 6900,-45.668,-73.299,32,225,228,...,79.088,231.137,30.0687,23.1375,NaN,NaN,X,32,20:23,12
6460,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,2-10-2022,2-10-2022,CTD SeaBird SBE 19plus V 2.5.2 SERIAL NO. 6900,-45.668,-73.299,32,226,228,...,78.721,230.065,30.069,23.1378,NaN,NaN,X,32,20:23,12
6461,ESTUDIO SEGUIMIENTO DESEMPEÑO AMBIENTAL ACUICU...,656-150,2-10-2022,2-10-2022,CTD SeaBird SBE 19plus V 2.5.2 SERIAL NO. 6900,-45.668,-73.299,32,227,228,...,78.86,230.471,30.069,23.1378,NaN,NaN,X,32,20:23,12


In [1]:
def limpieza_ctd(df):

    try:

        #eliminamos las filas innecesarias 
        df = df.iloc[20: , :]

        #Reiniciamos los indices
        df = df.reset_index(drop = True)

        #Realizamos una copia del dataset
        df_ctd_copy = df.copy()

        #Dejamos la primera fila como columna 

        df_ctd_copy.columns = df_ctd_copy.iloc[0]

        #Eliminamos la primera fila del dataset
        df_ctd_copy = df_ctd_copy.iloc[1: , :]

        #Reiniciamos los indices
        df_ctd_copy =  df_ctd_copy.reset_index(drop= True)

        #Agregamos la columna ID 
        df_ctd_copy.insert(0 , 'ID' , df_ctd_copy.index + 1)

        #Creamos la lista de columnas 
        columnas = df_ctd_copy.columns.to_list()

        # Modificamos las columnas de estación ya que existen dos de ellas

        df_ctd_copy.columns.values[8] = 'ESTACION_1'
        df_ctd_copy.columns.values[21] = 'ESTACION_2'

        #Eliminamos la estacion_2 ya que es innecesaria
        df_ctd_copy.drop(columns=['ESTACION_2'], inplace=True)

        # Renombramos la ESTACION_1 por ESTACION

        df_ctd_copy.columns.values[8] = 'ESTACION'

        #Creamos la lista de columnas 
        columnas = df_ctd_copy.columns.to_list()
    

    	#Creamos las columnas fijas y variables

        columnas_fijas = ['ID', 'NOMB_PROY', 'COD_PROY', 'FECHA_INI', 'FECHA_TER', 'EQUIPO', 'LATITUD', 'LONGUITUD', 'ESTACION', 'PROF_EQ', 'PROF_SECT' , 'HORA_INICIO' , 'COD_REG']
        columnas_variables = [ 'TEMPERATUR', 'OXIG_ml/L', 'OXIG_mg/L', 'OXIG_%sat', 'OX_umol/kg','SALINIDAD', 'DENSIDAD', 'CLOROFILA', 'FEOPIGMEN' , 'D_SECCHI']


        #Realizamos un melt para convertir las columnas_variables en filas 

        df_ctd_copy = pd.melt(df_ctd_copy , id_vars= columnas_fijas , value_vars= columnas_variables , var_name= 'VARIABLE' , value_name='VALOR')


        #Vamos a reemplazar los valores e HORA_INICIO ya que están mal escritos

        
        def normalizar_hora_string(hora_str):
            """
            Normaliza una cadena de tiempo para asegurar que los minutos tengan dos dígitos.
            Ej: '12:5' se convierte en '12:05'.
            """
            if pd.isna(hora_str): # Maneja posibles valores NaN/nulos si los hubiera
                return hora_str

            partes = str(hora_str).split(':')
            if len(partes) != 2:
                # Manejar casos donde el formato no es 'HH:MM' (ej. ya está mal, o es un dato inesperado)
                # Puedes decidir si quieres levantar un error, devolver el original, o un valor específico.
                # Por simplicidad, devolveremos el original si el formato no es el esperado de dos partes.
                return hora_str

            horas = partes[0]
            minutos = partes[1]

            if len(minutos) == 1:
                minutos = '0' + minutos # Añadir el cero delante si es un solo dígito

            return f"{horas}:{minutos}"
        
        #Aplicamos la funcion de hora a la columnas HORA_INICIO

        df_ctd_copy['HORA_INICIO'] = df_ctd_copy['HORA_INICIO'].apply(normalizar_hora_string)


        # Creamos una funcion para convertir la columna de HORA_INICIO a datetime para luego Crear una columna de fecha y hora 
        def convertir_hora(time_value):
            if isinstance(time_value, str): 
                # Separamos el ":" del texto y lo convertimos a entero y obtenemos dos variables
                horas, minutos = map(int, time_value.split(':'))
                #Retornamos los valores de Horas y minutos
                return datetime.time(horas, minutos)
            else:
                # En caso que ya es datetime.time se deja como esta
                return time_value


        df_ctd_copy['HORA_INICIO'] = df_ctd_copy['HORA_INICIO'].apply(convertir_hora)


         #Cambiamos los tipos de FECHA_INI y FECHA_FIN a datetime

        df_ctd_copy['FECHA_INI'] = pd.to_datetime(df_ctd_copy['FECHA_INI'])
        df_ctd_copy['FECHA_TER'] = pd.to_datetime(df_ctd_copy['FECHA_TER'])

        #FECHA INI CONVERTIDA A STRING 
        df_ctd_copy['FECHA_INI_STR'] =  df_ctd_copy['FECHA_INI'].dt.strftime("%d/%m/%Y")
        df_ctd_copy['FECHA_FIN_STR'] = df_ctd_copy['FECHA_TER'].dt.strftime("%d/%m/%Y")

        
        #Creamos una funcion lambda para combinar FECHA_INI con la HORA_INI
        df_ctd_copy['FECHA_INICIO'] = df_ctd_copy.apply(
        lambda row: datetime.datetime.strptime(row['FECHA_INI_STR'], "%d/%m/%Y").replace(
            hour=row['HORA_INICIO'].hour,
            minute=row['HORA_INICIO'].minute,
            second=row['HORA_INICIO'].second
            ),
            axis=1
        )


        return df_ctd_copy , columnas , columnas_fijas  , columnas_variables

    except Exception as e:
        print(f"Ocurrió un error en la función limpieza_macrofauna: {e}")
        ex = sys.exc_info()[2]
        print("Error en el archivo : " + str(ex.tb_frame) + " En la linea : " + str(ex.tb_lineno))
        
    return None #O devuelve un valor que indique el fallo


In [2]:
df_ctd_copy , columnas , columnas_fijas , columnas_variables = limpieza_ctd(df_ctd)

NameError: name 'df_ctd' is not defined

In [15]:
df_ctd_copy.columns

Index(['ID', 'NOMB_PROY', 'COD_PROY', 'FECHA_INI', 'FECHA_TER', 'EQUIPO',
       'LATITUD', 'LONGUITUD', 'ESTACION', 'PROF_EQ', 'PROF_SECT',
       'HORA_INICIO', 'COD_REG', 'VARIABLE', 'VALOR', 'FECHA_INI_STR',
       'FECHA_FIN_STR', 'FECHA_INICIO'],
      dtype='object')